<a href="https://colab.research.google.com/github/yasntrk/IBM_SpeechToSpeech/blob/master/SpeechToSpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
!pip install ibm_watson

In [ ]:
pip install wavio

In [ ]:
pip install pydub

In [ ]:
pip install playsound

In [ ]:
!pip install ffmpeg-python

In [ ]:
from ibm_watson import SpeechToTextV1, LanguageTranslatorV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import os 
import wavio as wv
from pydub import AudioSegment
from pydub import AudioSegment
from pydub.playback import play
from playsound import playsound
from IPython.display import Audio
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg


# API Keys

In [ ]:
ltapikey = "xx"
lturl = "xx"

sttapikey = 'xx'
stturl = 'xx'

txtsp = "xx"
txturl = "xx"

# Voice Record and Converting

In [ ]:

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
audio, sr = get_audio()

In [ ]:
sound_name = "Hey.wav"
wv.write(sound_name, audio, 44100, sampwidth=2)
os.path.join("/content", sound_name)

'/content/Hey.wav'

# Speech To Text

In [ ]:
ltauthenticator = IAMAuthenticator(ltapikey)
lt = LanguageTranslatorV3(version='2018-05-01', authenticator=ltauthenticator)
lt.set_service_url(lturl)

In [ ]:
sttauthenticator = IAMAuthenticator(sttapikey)
stt = SpeechToTextV1(authenticator=sttauthenticator)
stt.set_service_url(stturl)

In [ ]:
AudioSegment.from_wav("Hey.wav").export("Heyo.mp3", format="mp3")

<_io.BufferedRandom name='Heyo.mp3'>

In [ ]:
with open("Heyo.mp3", 'rb') as f:
    res = stt.recognize(audio=f, content_type='audio/mp3', model='en-US_NarrowbandModel', continuous=True).get_result()

In [ ]:
res

{'result_index': 0,
 'results': [{'alternatives': [{'confidence': 0.85,
     'transcript': "hi how are you I hope you're doing good I miss you please come back later "}],
   'final': True}]}

In [ ]:
voicetext = res['results'][0]['alternatives'][0]['transcript']
voicetext

"hi how are you I hope you're doing good I miss you please come back later "

In [ ]:
german = 'en-de'

In [ ]:
translation = lt.translate(text=voicetext, model_id=german).get_result()
translatedtext = translation['translations'][0]['translation']
translatedtext

'hi wie bist du ich hoffe ihr tut gut ich vermisse sie bitte kommen später '

In [ ]:
file = open("translated.txt", "w") 
file.write(translatedtext) 
file.close() 

In [ ]:
with open('translated.txt', 'r') as f:
    text = f.readlines()

In [ ]:
text = [line.replace('\n','') for line in text]

In [ ]:
text = ''.join(str(line) for line in text)

# Text To Speech


In [ ]:
from ibm_watson import TextToSpeechV1

In [ ]:
# Setup Service
authenticator = IAMAuthenticator(txtsp)
tts = TextToSpeechV1(authenticator=authenticator)
tts.set_service_url(txturl)

In [ ]:
with open('./translated_voice.mp3', 'wb') as audio_file:
    res = tts.synthesize(text, accept='audio/mp3', voice="de-DE_BirgitV3Voice").get_result()
    audio_file.write(res.content)

In [ ]:
with open('result.txt', 'w') as f:
    f.write(translatedtext)

In [ ]:
translated_file = open("translated.txt", "r") 
translated_file

<_io.TextIOWrapper name='translated.txt' mode='r' encoding='UTF-8'>

In [ ]:
translated_file.read()

'hi wie bist du ich hoffe ihr tut gut ich vermisse sie bitte kommen später '

In [ ]:
Audio("/content/translated_voice.mp3",rate=44100)